In [308]:
import pandas as pd
import numpy as np

redcom_df = pd.read_csv('Social Media - Redcomm.csv')

redcom_df.head()

,id,ingestion_date,link,date,post_message,like,comments,saved,share,engagement,video_views,reach,impression,type,username
0,8495879833,25 May 2023,https://www.instagram.com/p/Co4wPXXJjku/,2/20/23 14:00,"Serial yang diangkat dari kisah nyata, tentang...","10,865",71,15.0,134.0,"11,085","1,424","2,372,168","3,392,200",Reels,Netflixid
1,8495879833,25 May 2023,https://www.instagram.com/p/Co4ihdsJevl/,2/20/23 12:00,"?? Begini nasib, jadi third wheeler~","2,588",21,26.0,91.0,"2,726","1,739","466,859","840,346",Videos,Netflixid
2,8495879833,25 May 2023,https://www.instagram.com/reel/Co4UwKhp1uw/,2/20/23 10:00,"Udah akrab gini, apakah Jae-woo bakal berujung...","1,494",14,113.0,67.0,"1,688",0,"190,069","334,521",Picture,Netflixid
3,8495879833,25 May 2023,https://www.instagram.com/reel/Co4HBkaJm_Y/,2/20/23 8:00,Diajak main Alphabet Challenge pake huruf D-E-...,"1,271",10,39.0,NaN,"1,405","1,409","314,446","455,947",Reels,Netflixid
4,8495879833,25 May 2023,https://www.instagram.com/p/Co35ZDtJYai/,2/20/23 6:00,Besok kita buka bareng-bareng gaes suratnya!,"19,829",412,91.0,117.0,"20,449",0,"1,518,338","2,307,874",Carousels,Netflixid


# **CLEANSING**

## DUPLICATE DETECTION

In [332]:
print(f'Rows before remove duplicate : {len(redcom_df)}')
print(f'Total rows duplicate         : {redcom_df.duplicated().sum()}')
redcom_df.drop_duplicates(inplace=True)
print(f'Rows after remove duplicate  : {len(redcom_df)}')

Rows before remove duplicate : 2613
Total rows duplicate         : 15
Rows after remove duplicate  : 2598


## NULL DETECTION

In [309]:
redcom_df.isna().sum()

id                   0
ingestion_date       0
link                 0
date                 1
post_message      2328
like                 2
comments             4
saved                7
share                5
engagement           1
video_views          0
reach                0
impression           0
type                 0
username             0
dtype: int64

In [310]:
# FILL POST MESSAGE
redcom_df['post_message'].fillna('-',inplace=True)

In [311]:
# FILL DATE
redcom_df['date'] = redcom_df['date'].fillna('3/22/22 17:04')

In [312]:
def removeComma(x):
    return x.str.replace(',','')

def convertFloat(x):
    #x = x.str.replace(',','')
    return pd.to_numeric(x)

In [313]:
for i in ['like','comments','engagement','video_views','reach','impression']:
    redcom_df[i] = removeComma(redcom_df[i])
    redcom_df[i] = convertFloat(redcom_df[i])

In [314]:
#FILL SOCMED MATRIX LIKE, COMMENTS, SAVED, SHARE, ENGAGEMENT

redcom_df['like'] = redcom_df.apply(
    lambda row: row['engagement'] - row['share'] - row['comments'] - row['saved'] if np.isnan(row['like']) else row['like'],
    axis=1)

redcom_df['comments'] = redcom_df.apply(
    lambda row: row['engagement'] - row['like'] - row['share'] - row['saved'] if np.isnan(row['comments']) else row['comments'],
    axis=1)

redcom_df['saved'] = redcom_df.apply(
    lambda row: row['engagement'] - row['like'] - row['comments'] - row['share'] if np.isnan(row['saved']) else row['saved'],
    axis=1)

redcom_df['share'] = redcom_df.apply(
    lambda row: row['engagement'] - row['like'] - row['comments'] - row['saved'] if np.isnan(row['share']) else row['share'],
    axis=1)

redcom_df['engagement'] = redcom_df.apply(
    lambda row: row['share'] + row['like'] + row['comments'] + row['saved'] if np.isnan(row['engagement']) else row['engagement'],
    axis=1)

In [315]:
redcom_df.isna().sum()

id                0
ingestion_date    0
link              0
date              0
post_message      0
like              0
comments          0
saved             0
share             0
engagement        0
video_views       0
reach             0
impression        0
type              0
username          0
dtype: int64

## TIPE DATA

In [316]:
redcom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              2613 non-null   int64  
 1   ingestion_date  2613 non-null   object 
 2   link            2613 non-null   object 
 3   date            2613 non-null   object 
 4   post_message    2613 non-null   object 
 5   like            2613 non-null   float64
 6   comments        2613 non-null   float64
 7   saved           2613 non-null   float64
 8   share           2613 non-null   float64
 9   engagement      2613 non-null   float64
 10  video_views     2613 non-null   int64  
 11  reach           2613 non-null   int64  
 12  impression      2613 non-null   int64  
 13  type            2613 non-null   object 
 14  username        2613 non-null   object 
dtypes: float64(5), int64(4), object(6)
memory usage: 306.3+ KB


In [317]:
redcom_df['date'] = pd.to_datetime(redcom_df['date'])
redcom_df['ingestion_date'] = pd.to_datetime(redcom_df['ingestion_date'])

redcom_df['tanggal_post'] = pd.to_datetime(redcom_df['date'], format='%m/%d/%y')
redcom_df['tanggal_post'] = redcom_df['tanggal_post'].dt.to_period('D')
redcom_df['waktu_post'] = redcom_df['date'].dt.strftime('%X')
redcom_df['waktu_post'] = redcom_df['waktu_post'].str.slice(0,2)

/var/folders/tp/91my4k8n68z60sh31dbxcf1r0000gn/T/ipykernel_5239/764453062.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  redcom_df['date'] = pd.to_datetime(redcom_df['date'])


In [319]:
for i in ['waktu_post']:
    redcom_df[i] = convertFloat(redcom_df[i])
    
redcom_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2613 entries, 0 to 2612
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              2613 non-null   int64         
 1   ingestion_date  2613 non-null   datetime64[ns]
 2   link            2613 non-null   object        
 3   date            2613 non-null   datetime64[ns]
 4   post_message    2613 non-null   object        
 5   like            2613 non-null   float64       
 6   comments        2613 non-null   float64       
 7   saved           2613 non-null   float64       
 8   share           2613 non-null   float64       
 9   engagement      2613 non-null   float64       
 10  video_views     2613 non-null   int64         
 11  reach           2613 non-null   int64         
 12  impression      2613 non-null   int64         
 13  type            2613 non-null   object        
 14  username        2613 non-null   object        
 15  tang

## ANOMALI CATEGORICAL

In [322]:
for i in ['type']:
    display(i, redcom_df[i].unique())

'type'

array(['Reels', 'Videos', 'Picture', 'Carousels'], dtype=object)